In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image

In [ ]:
#8 MIN TO RUN
def load_data():
    # Load training images and labels
    train_images = []
    train_labels = []
    train_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/images'
    train_label_file = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/labels'
    for filename in os.listdir(train_dir):
        img = Image.open(os.path.join(train_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        train_images.append(img_arr)

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(train_label_file, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            train_labels.append(label)

    # Load testing images and labels
    test_images = []
    test_labels = []
    test_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/images'
    test_label_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/labels'
    for filename in os.listdir(test_dir):
        img = Image.open(os.path.join(test_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        test_images.append(img_arr)

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(test_label_dir, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            test_labels.append(label)

    # Convert the lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, test_images, train_labels, test_labels

# Load the dataset
X_train, X_test, y_train, y_test = load_data()

print("Number of training images: ", len(X_train))
print("Number of training labels: ", len(y_train))
print("Number of testing images: ", len(X_test))
print("Number of testing labels: ", len(y_test))

print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

# Print information about the dataset
print("Number of training images: ", X_train.shape[0])
print("Number of training labels: ", y_train.shape[0])
print("Number of testing images: ", X_test.shape[0])
print("Number of testing labels: ", y_test.shape[0])

Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458
Shape of X_train:  (2394, 224, 224, 3)
Shape of y_train:  (2394,)
Shape of X_test:  (458, 224, 224, 3)
Shape of y_test:  (458,)
Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458


In [ ]:
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Extract HOG features from training data
hog_train = []
for img in X_train:
    hog_train.append(hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), channel_axis=-1))

# Extract HOG features from test data
hog_test = []
for img in X_test:
    hog_test.append(hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), channel_axis=-1))

# Load training and testing data for all three classes
X_all = np.concatenate((X_train, X_test))
y_all = np.concatenate((y_train, y_test))

# Extract HOG features from all data
hog_all = []
for img in X_all:
    hog_all.append(hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), channel_axis=-1))

# Train SVM classifier with best hyperparameters
clf = SVC(kernel='rbf', C=0.1, gamma=0.1)
clf.fit(hog_train, y_train)

# Predict on test data
y_pred = clf.predict(hog_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

# Print classification report
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Accuracy:  0.8471615720524017

Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.85      1.00      0.92       388
           2       0.00      0.00      0.00        19

    accuracy                           0.85       458
   macro avg       0.28      0.33      0.31       458
weighted avg       0.72      0.85      0.78       458

Confusion Matrix:

[[  0  51   0]
 [  0 388   0]
 [  0  19   0]]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import joblib
import os

# Define directory path to save the joblib file
directory = '/content/drive/MyDrive/CW_Folder_UG/Models'

# Create directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Define file path to save the joblib file
filepath = os.path.join(directory, 'hog-svm.joblib')

# Dump the object to the joblib file
joblib.dump(clf, filepath)



['/content/drive/MyDrive/CW_Folder_UG/Models/hog-svm.joblib']

In [ ]:
import joblib
import os

# create folder if it doesn't exist
folder_path = os.path.join('content', 'drive', 'MyDrive', 'CW_Folder_UG', 'Models')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# save model in folder
file_path = os.path.join(folder_path, 'hog-svm.joblib')
joblib.dump(clf, file_path)



['content/drive/MyDrive/CW_Folder_UG/Models/hog-svm.joblib']

In [ ]:
print(os.listdir(folder_path))


['hog-svm.joblib']
